In [2]:
# API_KEY: kOCHMwXaZkrC5ov6AcLslH4ZwqjP0won
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from datetime import date, datetime, timedelta
import time

import tensorflow_addons as tfa

import keras
import joblib

In [19]:
# Create an instance of Real time prediction class
stockName = 'AMD'
params = ['open', 'high', 'low', 'close', 'volume', 'V1', 'V2', 'V3']
modelPath = 'model_03_23_3.h5'
# modelPath = '/content/drive/MyDrive/03_10_Stock_Price_Prediction/Saved_Models/model11.h5'
scalerPath = 'scaler_amd1year_3.save'
model = keras.models.load_model(modelPath)
scaler = joblib.load(scalerPath)
diffClose = 1

c:\Users\IAMAI\miniconda3\envs\env_3_7_8\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [20]:
filePath = 'test.csv'
df = pd.read_csv(filePath)

In [21]:
df = df.rename(columns={'t': 'time stamp', 'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'})
df.columns

Index(['symbol', 'time stamp', 'open', 'high', 'low', 'close', 'volume', 'vw',
       'V1', 'V2', 'V3'],
      dtype='object')

In [22]:
def addAdditionalFeatures(df):
    """This function calculates additional parameters"""
    df_new = pd.DataFrame(columns = ['time stamp', 'open', 'high', 'low', 'close', 'volume', 'V1', 'V2', 'V3'])
    for i in range(len(df)):
        tmpList = df.iloc[i][['time stamp', 'open', 'high', 'low', 'close', 'volume']].values.tolist()
        tmpList.append(df.iloc[i]['high'] - df.iloc[i]['low'])
        tmpList.append(df.iloc[i]['close'] - df.iloc[i]['open'])
        if i < 10:
            tmpList.append(0)
        else:
            tmpList.append(df.iloc[i]['volume'] - df['volume'].iloc[i - 10:i - 1].sum() / 10)
        df_new.loc[i] = tmpList
    return df_new

In [23]:
df_new = addAdditionalFeatures(df)
df_new

,time stamp,open,high,low,close,volume,V1,V2,V3
0,3/16/2023 7:09,89.80,89.80,89.80,89.80,850,0.00,0.00,0
1,3/16/2023 7:10,89.83,89.83,89.80,89.80,779,0.03,-0.03,0
2,3/16/2023 7:11,89.77,89.77,89.77,89.77,895,0.00,0.00,0
3,3/16/2023 7:12,89.80,89.82,89.80,89.82,1497,0.02,0.02,0
4,3/16/2023 7:13,89.76,89.76,89.76,89.76,981,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...
3004,3/21/2023 19:55,95.30,95.30,95.30,95.30,512,0.00,0.00,-544.9
3005,3/21/2023 19:56,95.35,95.35,95.35,95.35,480,0.00,0.00,-654.4
3006,3/21/2023 19:57,95.35,95.40,95.35,95.40,1295,0.05,0.05,183.4
3007,3/21/2023 19:58,95.35,95.35,95.30,95.30,5704,0.05,-0.05,4643.7


In [24]:
test = scaler.transform(df_new[params])

In [25]:
"""Define the function to create 
the input and output dataset for the training, validation and test"""
def create_dataset(transformed, df, prevMin, predMin):
    Xs, ys = [], []
    for i in range(len(transformed) - prevMin - predMin):
        vx = []
        for j in range(prevMin):
            vx.append(transformed[i + j])
        # vy = []
        # for j in range(predMin):
        #     vy.append(df['close'].iloc[i + j + prevMin])
        vy = 0
        diff = df['close'].iloc[i + predMin] - df['close'].iloc[i]
        if diff > diffClose:
          vy = 2
        elif diff < -diffClose:
          vy = 0
        else:
          vy = 1
        vx = np.array(vx)
        vy = np.array(vy)
        vx = vx.reshape(-1, 1)
        Xs.append(vx)
        ys.append(vy)
    return np.array(Xs), np.array(ys)

In [26]:
test_X, test_y = create_dataset(test, df_new, 120, 60)

In [33]:
pred = model.predict(test_X)

89/89 [==============================] - 20s 202ms/step


In [37]:
s = 0
for i in range(test_X.shape[0]):
    print(np.argmax(pred[i]), test_y[i])
    s += (np.argmax(pred[i]) == test_y[i])
print(s / i)

1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 2
2 2
2 2
2 2
2 2
1 1
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
1 2
2 2
2 2
1 2
2 2
2 2
1 1
1 1
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
1 2
1 2
1 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
0 1
0 1
0 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
